#  Iterative phase estimation

|||
|-|-|
|**Authors:** |Taha Selim, Alain Chancé|
|**Date:** |October 21, 2023|
|**Version:** |**1.00**<br/>*Details see at the end of this notebook*|
|**References:**|
[Prof. Gerrit C. Groenenboom, Quantum theoretical chemistry (NWI-MOL112), May 26, 2023](https://www.theochem.ru.nl/ctc2/pdf/qtclecture.pdf)
[Prof. Gerrit C. Groenenboom, Computational and Theoretical Chemistry 2, NWI-MOL176 (3EC)](https://www.theochem.ru.nl/ctc2/)
[Keeper L. Sharkey, Alain Chancé, Quantum Chemistry and Computing for the Curious: Illustrated with Python and Qiskit® code, ISBN-13: 978-1803243900](https://www.amazon.com/Quantum-Chemistry-Computing-Curious-Illustrated/dp/1803243902/)

The following MIT license only applies to the code, and not to the text and images.

# MIT License

Copyright (c) 2023 Taha Selim, Alain Chancé

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [1]:
using Plots # or StatsPlots
using LinearAlgebra
using SpecialFunctions
using AssociatedLegendrePolynomials
using LaTeXStrings
using Quantikz

## Include files
### Include lib_load.jl which loads common modules with includes

In [1]:
# import conventions
include("conventions.jl")
using .conventions: big_endian, qubit_begin, little_endian

# import quantum gates
include("quantum_gates.jl")
using ..quantum_gates: Qgate, Rz_gate1

include("lib_tensor/QTensor.jl")
using ..QTensor: Qgate_T2D

include("lib_useful/custom_functions.jl")
using ..custom_functions: MK_sortrows

include("quantum_circuit.jl")
using ..quantum_circuit: qc_init, init_register, show_statevector, op
#using ..quantum_circuit: qc_init, init_register, print_initstate

UndefVarError: UndefVarError: `little_endian` not defined

# Qubits and quantum gates
## Qubits
A qubit is a quantum unit of information that represents a two-level quantum system and lives in a two-dimensional Hilbert space $\mathbb C^2$. The computational basis states of the quantum space are denoted as $\{|0\rangle,|1\rangle\}$:

$$|0\rangle=\left(\begin{array}{l}
1 \\
0
\end{array}\right) 
|1\rangle=\left(\begin{array}{l}
0 \\
1
\end{array}\right)$$

Any single-qubit state is described by a linear superposition of the computational basis with complex coefficients:  
$$|\psi\rangle=\alpha|0\rangle+\beta|1\rangle=\left(\begin{array}{l}
\alpha \\
\beta
\end{array}\right) \in \mathbb{C}^{2}$$

where 𝛼 and 𝛽 satisfy:
$$|\alpha|^{2}+|\beta|^{2}=1$$

A qubit is in a quantum superposition during the execution of an algorithm.  When it is measured in the computational basis, a qubit will be found in either state $|0\rangle$ or in state $|1\rangle$ with probability $|\alpha|^{2}$ and $|\beta|^{2}$ respectively. If there are $n$ qubits in the system, the state is described by a vector in the $2^n$ dimensional Hilbert space $(\mathbb{C}^2)^{⊗n}$ formed by taking the tensor product of the Hilbert spaces of the individual qubits. For example, for 10 qubits, the state is described by a vector in a 1024-dimensional Hilbert space. 

## Tensor ordering of qubits
The physics community typically orders a tensor product of $n$ qubits with the first qubit on the left-most side of the tensor product: 

$$|q\rangle=\left|q_{0}\right\rangle\left|q_{1}\right\rangle \ldots\left|q_{n-1}\right\rangle=\left|q_{0}, q_{1}, \ldots, q_{n-1}\right\rangle=\bigotimes_{i=0}^{n-1}\left|q_{i}\right\rangle$$

However, Qiskit uses an ordering in which the $n^{th}$ qubit is first in the order and the $0^{th}$ qubit is last:

$$|q\rangle=
\left|q_{n-1}\right\rangle\ldots\left|q_{1}\right\rangle\left|q_{0}\right\rangle
=\left|q_{n-1}, \ldots, q_{1}, q_{0}\right\rangle
=\bigotimes_{i=n-1}^{0}\left|q_{i}\right\rangle$$

In other words, if qubit $0$ is in state $|0\rangle$, qubit $1$ is in state $|0\rangle$, and qubit 2 is in state $|1\rangle$, the state represented in many physics textbooks as $|001\rangle$ is represented by Qiskit as $|100\rangle$. This difference affects the way multi-qubit operations are represented as matrices.

## Single qubit quantum gates
A single qubit quantum gate $U$ has a $(2\times2)$ unitary matrix form: $U^\dagger U=UU^\dagger= 1$.

### X Gate
An X gate maps $|0\rangle$ to $|1\rangle$ and $|1\rangle$ to $|0\rangle$. For classical computing, the NOT gate changes a 0 to a 1 and a 1 to a 0.

$$X = \begin{pmatrix}
0 & 1 \\
1 & 0 \\
\end{pmatrix}$$

### H Gate
A Hadamard gate maps the basis state $|0\rangle$ to $\frac{|0\rangle + |1\rangle}{\sqrt{2}}$ which is also written as $|+\rangle$ and $|1\rangle$ to $\frac{|0\rangle - |1\rangle}{\sqrt{2}}$ which is also written as $|-\rangle$. A measurement of the state $|+\rangle$ or of the state $|-\rangle$ will have equal probabilities of being $0$ or $1$, creating a superposition of states.

$$H = \frac{1}{\sqrt{2}}\begin{pmatrix}
1 & 1 \\
1 & -1 \\
\end{pmatrix}$$

## Two qubits quantum gates
A two qubits gate $U$ has a 4x4 unitary matrix form, $U^\dagger U=UU^\dagger= 1$.

### Controlled Not (CNOT, CX) Gate

If the first qubit is |1⟩ it performs the Pauli-X (NOT) operation on the second qubit, otherwise it leaves it unchanged

With the tensor ordering of qubits used in most physics textbooks:

$$CX = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 \end{pmatrix}$$

With Qiskit tensor ordering of qubits:

$$CX = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0 \\
\end{pmatrix}$$

## Quantum Phase Estimation (QPE)
In quantum chemistry we need very accurate calculations of the total electronic energy of each molecule species involved in a chemical reaction [Burg]. The Quantum Phase Estimation (QPE) algorithm has a unique feature that it allows a bounded-error simulation of quantum systems which makes it one of the most promising application of future fault tolerant quantum computing. Given a unitary operator $U$, its eigenstate and eigenvalues, $U|\psi\rangle=e^{2 \pi i \theta}|\psi\rangle$, the ability to prepare a state $|\psi\rangle$ and the ability to apply $U$ itself, the QPE algorithm calculates $2^n θ$ where $n$ is the number of qubits used to estimate $\theta$ thereby allowing measurement of $\theta$ as precise as we want.

Recall that in Section 2.5, Postulate 5 – Time evolution dynamics, we saw that time evolution dynamics of a quantum system is described by the Schrödinger’s equation:

$$i \hbar \frac{d}{d t}|\psi\rangle=\hat{H}|\psi\rangle$$

For a time independent Hamiltonian $\hat{H}$ with initial condition$\left|\psi\left(t_{0}\right)\right\rangle$ the solution is:

$$|\psi(t)\rangle=U(t)\left|\psi\left(t_{0}\right)\right\rangle$$

where $U(t)=\exp \left(-i \frac{t}{\hbar} \hat{H}\right)$ is the unitary time-evolution operator. Further recall that any unitary matrix has eigenvalues of the form $e^{i \theta}$. An eigenvalue of $U(t)$ is also an eigenvalue of $\hat{H}$.

First we define a function $U(\theta)$ which creates a quantum circuit with a single qubit $\left|q_{0}\right\rangle$ and applies the following unitary:

$$U(\theta)\left|q_{0}\right\rangle=e^{2 \pi i \theta}\left|q_{0}\right\rangle=p(2 \pi \theta)\left|q_{0}\right\rangle$$

where $p(\lambda)$ is the gate which has the matrix form:

$$p(λ)= \begin{pmatrix}
1 & 0 \\
0 & e^{i\lambda} \\
\end{pmatrix}$$

### Define the function U_op() which applies the unitary operator whose eigenvalue will be measured

In [2]:
function U_op(theta::Float64, endian=little_endian)
    qc = qc_init(2, endian)
    p = Qgate.P(pi*2*theta)
    p = Qgate_T2D(p, 0, 2, endian)
    op(qc,p)
    return qc
end

U_op (generic function with 2 methods)

In [7]:
p = Qgate.P(pi*2*theta)
p = Qgate_T2D(p, 0, 2, little_endian)

4×4 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im       0.0+0.0im            0.0+0.0im
 0.0+0.0im  1.0+0.0im       0.0+0.0im            0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.707107-0.707107im       0.0-0.0im
 0.0+0.0im  0.0+0.0im       0.0-0.0im       0.707107-0.707107im

In [8]:
theta = 1/2 + 1/4 + 1/8
u = U_op(theta)

Main.quantum_circuit.qc_initstruct(2, "little-endian", 2, 4, ComplexF64[1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im], [0.0 0.0; 1.0 0.0; 0.0 1.0; 1.0 1.0], false, false)

In [9]:
show_statevector(u)

1.0 + 0.0im * | [0, 0]>
0.0 + 0.0im * | [0, 1]>
0.0 + 0.0im * | [1, 0]>
0.0 + 0.0im * | [1, 1]>


### Define the function construct_circuit() which builds the $k^{th}$ iteration Quantum Phase Estimation circuit.

Arguments:
- u: The circuit representing the unitary operator whose eigenvalue (via phase) will be measured and one auxiliary qubit
- k: the iteration index.
- omega: the feedback angle.
- endian: big_endian or little_endian

Returns:
- u: the quantum circuit per iteration

In [6]:
function construct_circuit(u::Main.quantum_circuit.qc_initstruct, k::Int64, omega::Float64, endian=little_endian)
    
    Hgate = Qgate.H
    Hgate = Qgate_T2D(Hgate, 1, 2, endian)
    
    p = Qgate.U(0.0, 0.0, omega)
    p = Qgate_T2D(p, 0, 2, endian)
    
    C_U = Qgate.C_U(1, 0, 0.0, 0.0, 0.0, omega, endian)
    
    # Hadamard on circuit u qubit 1
    op(u, Hgate)
    
    # Apply 2^(k-1) times the Controlled-U operators
    r = 2^(k-1)
    println("r: ",r)
    
    for i in 1:r
        op(u, C_U)
    end
    
    # Apply phase gate with angle omega
    op(u, p)
    
    # Hadamard on circuit u qubit 1
    op(u, Hgate)
    
    return u
end

construct_circuit (generic function with 2 methods)

### Define the function estimate() which iteratively estimates the eigenphase of the input unitary and initial-state

Arguments:
- n_iter: The number of iterations
- u: The circuit representing the unitary operator whose eigenvalue (via phase) will be measured and one auxiliary qubit

Returns:
- Estimated phase

In [7]:
function estimate(n_iter::Int64, u::Main.quantum_circuit.qc_initstruct, endian=little_endian)
    
    Xgate = Qgate.X
    Xgate = Qgate_T2D(Xgate, 0, 2, endian)
    
    # X on circuit u qubit 0
    op(u, Xgate)
    
    omega_coef = 0
    
    for k in n_iter:-1:1
        omega_coef /= 2
        u = construct_circuit(u, k, -2*pi*omega_coef)
        
        # Simulate measurement 
        x = 1
        
        omega_coef = omega_coef + x/2
    end
    
    return omega_coef
end

estimate (generic function with 2 methods)

### Run the iterative phase estimation

In [8]:
theta = 1/2 + 1/4 + 1/8
u = U_op(theta)
estimate(3, u)

r: 4
r: 2
r: 1


0.875

In [9]:
theta = 1/2 + 1/4 + 1/8 + 1/16 + 1/32 + 1/64 + 1/128 + 1/256
u = U_op(theta)
estimate(8, u)

r: 128
r: 64
r: 32
r: 16
r: 8
r: 4
r: 2
r: 1


0.99609375

## References

[Groenenboom 1] Prof. Gerrit C. Groenenboom, Quantum theoretical chemistry (NWI-MOL112), May 26, 2023, https://www.theochem.ru.nl/ctc2/pdf/qtclecture.pdf

[Groenenboom 2] Prof. Gerrit C. Groenenboom, Computational and Theoretical Chemistry 2, NWI-MOL176 (3EC),https://www.theochem.ru.nl/ctc2/

[Sharkey] Sharkey, Keeper L., and Chancé, Alain. 2022. Quantum Chemistry and Computing for the Curious: Illustrated with Python and Qiskit Code. Packt, Pub. ISBN-13: 978-1803243900

[IPE] IterativePhaseEstimation, https://qiskit.org/ecosystem/algorithms/stubs/qiskit_algorithms.IterativePhaseEstimation.html

[IPE_Source] Source code for qiskit.algorithms.phase_estimators.ipe,
https://qiskit.org/documentation/stable/0.41/_modules/qiskit/algorithms/phase_estimators/ipe.html#IterativePhaseEstimation

[lab3] ibm-quantum-challenge-spring-2023, lab3.ipynb, https://github.com/qiskit-community/ibm-quantum-challenge-spring-2023/blob/main/content/lab_3/lab3.ipynb

[IPE_video] Iterative Quantum Phase Estimation | Qiskit Global Summer School 2023, https://www.youtube.com/watch?v=aLSM0_H8hUE

[DavitKhach] Iterative quantum phase estimation, https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/iterative_quantum_phase_estimation.ipynb

[Devesh] Implementing Quantum Phase Estimation Algorithm using Qiskit, https://medium.com/@deveshq/implementing-quantum-phase-estimation-algorithm-using-qiskit-e808e8167d32

## Display Julia version information

In [10]:
# Display Julia version information
versioninfo()

Julia Version 1.9.1
Commit 147bdf428cd (2023-06-07 08:27 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 4 × DO-Regular
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, haswell)
  Threads: 2 on 4 virtual cores
